# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[["CHAS", "RM", "B", "DIS", "LSTAT"]]
y = pd.DataFrame(boston.target, columns = ["target"])


## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [7]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [8]:
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print("Training Error: ", train_mse)
print("Testing Error: ", test_mse)

Training Error:  22.61147031945741
Testing Error:  18.303112171919686


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [ ]:
def kfolds(data, k):
    #Force data as pandas dataframe
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 

In [13]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers   
    return np.array_split(data, k)

In [14]:
data = pd.concat([X, y], axis=1)

In [15]:
data.head()

,CHAS,RM,B,DIS,LSTAT,target
0,0.0,6.575,1.000000,0.542096,-1.275260,24.0
1,0.0,6.421,1.000000,0.623954,-0.263711,21.6
2,0.0,7.185,0.989737,0.623954,-1.627858,34.7
3,0.0,6.998,0.994276,0.707895,-2.153192,33.4
4,0.0,7.147,1.000000,0.707895,-1.162114,36.2


In [16]:
data_folds = kfolds(data, 5)

In [17]:
len(data_folds)

5

In [18]:
for fold in data_folds:
    print(len(fold))

102
101
101
101
101


### Apply it to the Boston Housing Data

In [ ]:
# Make sure to concatenate the data again

In [19]:
#make fold 0 to be testing data, and fold 1-4 to be training data
n = 0
train_list = []
for counts, fold in enumerate(data_folds):
    if counts != n:
        train_list.append(fold)

df_train = pd.concat(train_list)
df_test = data_folds[n]


In [20]:
df_train.shape, df_test.shape

((404, 6), (102, 6))

In [21]:
df_train.head()

,CHAS,RM,B,DIS,LSTAT,target
102,0.0,6.405,0.177720,0.369415,-0.012136,18.6
103,0.0,6.137,0.993873,0.369415,0.378596,19.3
104,0.0,6.167,0.989384,0.321174,0.235000,20.1
105,0.0,5.851,0.992814,0.262627,0.717270,19.5
106,0.0,5.836,0.996898,0.282946,0.925237,19.5


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [26]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    train_list =[]
    # Split in train and test for the fold
    for counts, fold in enumerate(data_folds):
        if counts != n:
            train_list.append(fold)
    train = pd.concat(train_list)
    test = data_folds[n]
    # Fit a linear regression model
    linreg.fit(train[["CHAS", "RM", "B", "DIS", "LSTAT"]], train['target'])
    #Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[["CHAS", "RM", "B", "DIS", "LSTAT"]])
    train_errs.append(mean_squared_error(train['target'], y_hat_train))
    y_hat_test = linreg.predict(test[["CHAS", "RM", "B", "DIS", "LSTAT"]])
    test_errs.append(mean_squared_error(test['target'], y_hat_test))

print(train_errs)

print(test_errs)

[24.195577370388627, 23.032087348477972, 19.74507285797898, 15.317101384251194, 22.329972807546586]
[13.405144922008233, 17.444016800682032, 37.03271139002071, 58.279543847842326, 26.097988757148375]


In [27]:
np.mean(train_errs), np.mean(test_errs)

(20.92396235372867, 30.45188114354034)

In [ ]:
#test errors are much higher, which is an issue

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [30]:
np.mean(cv_5_results)

-30.451881143540312

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!